# Normalizing flows

Applying a function to a random variable turns it into another random variable. If the
function is invertible and differentiable, we can get the PDF of the transformed random
variable using the change of variables formula:
$$
p_Y(x) = \frac{p_X(z)}{\left| f'(z) \right|}
$$

where $z$ is the result of applying the inverse of the function ($z=f^{-1}(x)$).